##1. Загрузка пакетов - библиотек

In [23]:
# Импорт необходимых библиотек
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import random
from keras import models, layers
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Установка seed для воспроизводимости результатов
np.random.seed(17)
tf.random.set_seed(17)

##2. Загрузка датасета

In [24]:
dataset_root = Path("/content/iad-lab3/Vehicles")

# Проверка наличия датасета
if dataset_root.exists():
    print("Датасет уже найден по указанному пути.")
else:
    print("Датасет не найден. Выполняется загрузка с GitHub...")
    !git clone https://github.com/Daria-Chernykh/iad-lab3.git

# Получение списка классов (подкаталогов)
class_names = sorted(
    [item.name for item in dataset_root.iterdir() if item.is_dir()]
)

# Фиксация количества классов
num_classes = len(class_names)

print("\nНайденные классы:")
for idx, class_name in enumerate(class_names):
    print(f"{idx + 1}. {class_name}")

print(f"\nКоличество классов k = {num_classes}")

# Допустимые расширения изображений (в нижнем регистре)
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".webp"}

# Подсчёт количества изображений в каждой папке класса
print("\nКоличество изображений по классам:")

total_images = 0

for class_name in class_names:
    class_path = dataset_root / class_name

    num_images = sum(
        1 for f in class_path.iterdir()
        if f.is_file() and f.suffix.lower() in image_extensions
    )

    total_images += num_images
    print(f"{class_name}: {num_images}")

print(f"\nОбщее количество изображений в датасете: {total_images}")

Датасет уже найден по указанному пути.

Найденные классы:
1. Auto Rickshaws
2. Bikes
3. Cars
4. Motorcycles
5. Planes
6. Ships
7. Trains

Количество классов k = 7

Количество изображений по классам:
Auto Rickshaws: 800
Bikes: 800
Cars: 790
Motorcycles: 800
Planes: 800
Ships: 800
Trains: 800

Общее количество изображений в датасете: 5590


В лабораторной работе используется датасет изображений транспортных средств (Vehicles). Набор данных включает 7 классов:
* Авторикши (Auto Rickshaws),
* Велосипеды (Bikes),
* Машины (Cars),
* Мотоциклы (Motorcycles),
* Самолеты (Planes),
* Корабли (Ships),
* Поезда (Trains).

Каждый класс представлен отдельной папкой с изображениями. Общее количество изображений в датасете составляет 5590. Число изображений в классах близко по величине (от 790 до 800 изображений на класс), что позволяет считать датасет практически сбалансированным и пригодным для решения задачи многоклассовой классификации с использованием сверточных нейронных сетей.

##3. Выбор и фиксация размерности изображений

In [25]:
# Фиксация размерности входных изображений
img_height = 128
img_width = 128
input_shape = (img_height, img_width, 3)

# Размер мини-пакета
batch_size = 32

##4. Формирование обучающей, валидационной и тестовой выборок

####4.1 Физическое разбиение датасета по папкам

In [26]:
source_root = Path("/content/iad-lab3/Vehicles")
target_root = Path("/content/Vehicles_split")

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".webp"}

# Создание структуры каталогов
for split in ["train", "val", "test"]:
    for class_dir in source_root.iterdir():
        if class_dir.is_dir():
            (target_root / split / class_dir.name).mkdir(parents=True, exist_ok=True)

# Разбиение изображений по классам
for class_dir in source_root.iterdir():
    if not class_dir.is_dir():
        continue

    images = [
        img for img in class_dir.iterdir()
        if img.is_file() and img.suffix.lower() in image_extensions
    ]

    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    train_images = images[:n_train]
    val_images = images[n_train:n_train + n_val]
    test_images = images[n_train + n_val:]

    for img in train_images:
        shutil.copy(img, target_root / "train" / class_dir.name / img.name)

    for img in val_images:
        shutil.copy(img, target_root / "val" / class_dir.name / img.name)

    for img in test_images:
        shutil.copy(img, target_root / "test" / class_dir.name / img.name)

print("Разбиение датасета на train / val / test завершено.")


Разбиение датасета на train / val / test завершено.


####4.2 Создание генераторов изображений

In [27]:
# Генератор с аугментацией — только для обучающей выборки
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# Генераторы без аугментации — для валидации и теста
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    target_root / "train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

val_generator = val_test_datagen.flow_from_directory(
    target_root / "val",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = val_test_datagen.flow_from_directory(
    target_root / "test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


Found 5452 images belonging to 7 classes.
Found 3292 images belonging to 7 classes.
Found 3315 images belonging to 7 classes.


####4.3 Вывод количества изображений в каждой выборке

In [28]:
num_train = train_generator.samples
num_val = val_generator.samples
num_test = test_generator.samples

print("Количество изображений в выборках:")
print(f"Обучающая выборка: {num_train}")
print(f"Валидационная выборка: {num_val}")
print(f"Тестовая выборка: {num_test}")
print(f"Общее количество изображений: {num_train + num_val + num_test}")

Количество изображений в выборках:
Обучающая выборка: 5452
Валидационная выборка: 3292
Тестовая выборка: 3315
Общее количество изображений: 12059


Исходный датасет был физически разделён на обучающую, валидационную и тестовую выборки в соотношении 60% / 20% / 20% отдельно для каждого класса. Аугментация данных применялась только к обучающей выборке, что позволяет повысить обобщающую способность модели и избежать искажения оценок качества на валидационных и тестовых данных.

##5. Построение сверточной нейронной сети «с нуля» (Conv2D + MaxPooling + Dense)

In [29]:
# 1. СОЗДАНИЕ CALLBACK'ОВ
callbacks = [
    # Ранняя остановка при переобучении
    EarlyStopping(
        monitor='val_loss',
        patience=15,  # Количество эпох без улучшения
        restore_best_weights=True,
        verbose=1,
        mode='min'
    ),

    # Уменьшение learning rate при плато
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Уменьшение LR в 2 раза
        patience=7,  # Количество эпох без улучшения
        min_lr=1e-6,  # Минимальный learning rate
        verbose=1,
        mode='min'
    )
]

# 2. СЛОЙ АУГМЕНТАЦИИ
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

# 3. СОЗДАНИЕ МОДЕЛИ С АВТОКОДИРОВЩИКОМ
def create_autoencoder_classifier_model():
    """
    Создает комбинированную модель:
    - Энкодер для извлечения признаков
    - Два выхода: декодер (автокодировщик) и классификатор
    """

    inputs = layers.Input(shape=(128, 128, 3))

    # Аугментация данных
    x = data_augmentation(inputs)

    # Нормализация
    x = layers.Rescaling(1./255)(x)

    # ЭНКОДЕР (общая часть)
    # Блок 1
    x = layers.Conv2D(
        32, (3, 3),
        activation='relu',
        padding='same',
        kernel_regularizer=regularizers.l2(0.001)
    )(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.1)(x)

    # Блок 2
    x = layers.Conv2D(
        64, (3, 3),
        activation='relu',
        padding='same',
        kernel_regularizer=regularizers.l2(0.001)
    )(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.2)(x)

    # Блок 3
    x = layers.Conv2D(
        128, (3, 3),
        activation='relu',
        padding='same',
        kernel_regularizer=regularizers.l2(0.001)
    )(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # ДЕКОДЕР (для автокодировщика)
    # Начинаем декодирование от боттлнека
    # Блок 1 декодера
    d = layers.Conv2D(
        128, (3, 3),
        activation='relu',
        padding='same'
    )(d)
    d = layers.UpSampling2D((2, 2))(d)
    d = layers.Dropout(0.2)(d)

    # Блок 2 декодера
    d = layers.Conv2D(
        64, (3, 3),
        activation='relu',
        padding='same'
    )(d)
    d = layers.UpSampling2D((2, 2))(d)
    d = layers.Dropout(0.1)(d)

    # Блок 3 декодера
    d = layers.Conv2D(
        32, (3, 3),
        activation='relu',
        padding='same'
    )(d)
    d = layers.UpSampling2D((2, 2))(d)

    # Выход декодера (восстановленное изображение)
    decoder_output = layers.Conv2D(
        3, (3, 3),
        activation='sigmoid',
        padding='same',
        name='decoder_output'
    )(d)

    # КЛАССИФИКАТОР
    # Используем то же сжатое представление (encoded) для классификации
    c = layers.GlobalAveragePooling2D()(encoded)

    # Полносвязные слои
    c = layers.Dense(
        128,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    )(c)
    c = layers.Dropout(0.5)(c)

    c = layers.Dense(
        64,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    )(c)
    c = layers.Dropout(0.3)(c)

    # Выход классификатора
    classification_output = layers.Dense(
        7,
        activation='softmax',
        name='classification_output'
    )(c)

    # Создание модели с двумя выходами
    model = models.Model(
        inputs=inputs,
        outputs=[decoder_output, classification_output]
    )

    return model

# 4. СОЗДАНИЕ И КОМПИЛЯЦИЯ МОДЕЛИ
print("="*60)
print("СОЗДАНИЕ МОДЕЛИ")
print("="*60)

model = create_autoencoder_classifier_model()

# Компиляция с двумя loss функциями
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'decoder_output': 'mse',  # Mean Squared Error для автокодировщика
        'classification_output': 'categorical_crossentropy'  # Cross-entropy для классификации
    },
    loss_weights={
        'decoder_output': 0.3,  # Вес для автокодировщика
        'classification_output': 1.0  # Основной вес для классификации
    },
    metrics={
        'classification_output': ['accuracy']
    }
)

# model.summary()

# 5. ПОДГОТОВКА ГЕНЕРАТОРО
def multi_output_generator(generator):
    """
    Подготовка данных для модели с двумя выходами
    """
    for batch_x, batch_y in generator:
        yield batch_x, {
            'decoder_output': batch_x,  # Для автокодировщика: вход = выход
            'classification_output': batch_y  # Для классификатора: истинные метки
        }


СОЗДАНИЕ МОДЕЛИ


In [30]:
# 6. ОБУЧЕНИЕ МОДЕЛИ
print("\n" + "="*60)
print("ОБУЧЕНИЕ МОДЕЛИ")
print("="*60)

history = model.fit(
    multi_output_generator(train_generator),
    steps_per_epoch=max(1, num_train // batch_size),
    epochs=100,
    validation_data=multi_output_generator(val_generator),
    validation_steps=max(1, num_val // batch_size),
    callbacks=callbacks,
    verbose=1
)

# 7. СОЗДАНИЕ ОТДЕЛЬНОЙ МОДЕЛИ КЛАССИФИКАТОРА
print("\n" + "="*60)
print("СОЗДАНИЕ ЧИСТОЙ МОДЕЛИ КЛАССИФИКАТОРА")
print("="*60)

# Создаем модель только для классификации (без декодера)
classification_model = Model(
    inputs=model.input,
    outputs=model.get_layer('classification_output').output
)

# Компилируем только для классификации
classification_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# classification_model.summary()

# 8. ОЦЕНКА НА ТЕСТОВЫХ ДАННЫХ
print("\n" + "="*60)
print("ОЦЕНКА НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*60)

test_results = classification_model.evaluate(
    test_generator,
    steps=max(1, num_test // batch_size),
    verbose=1
)

print(f"\nТЕСТОВЫЕ МЕТРИКИ:")
print(f"Loss: {test_results[0]:.4f}")
print(f"Accuracy: {test_results[1]:.4f}")



ОБУЧЕНИЕ МОДЕЛИ
Epoch 1/100
 10/170 ━━━━━━━━━━━━━━━━━━━━ 31:27 12s/step - classification_output_accuracy: 0.1344 - classification_output_loss: 1.9461 - decoder_output_loss: 0.0882 - loss: 2.6433

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


130/170 ━━━━━━━━━━━━━━━━━━━━ 7:54 12s/step - classification_output_accuracy: 0.1322 - classification_output_loss: 1.9466 - decoder_output_loss: 0.0908 - loss: 2.2955

KeyboardInterrupt: 

In [ ]:
# 9. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
def plot_training_results(history):
    """
    Визуализация результатов обучения
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # График 1: Точность классификации
    axes[0, 0].plot(history.history['classification_output_accuracy'],
                   label='Обучающая', linewidth=2)
    axes[0, 0].plot(history.history['val_classification_output_accuracy'],
                   label='Валидационная', linewidth=2)
    axes[0, 0].set_title('Точность классификации', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Эпоха')
    axes[0, 0].set_ylabel('Точность')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # График 2: Потери классификации
    axes[0, 1].plot(history.history['classification_output_loss'],
                   label='Обучающие', linewidth=2)
    axes[0, 1].plot(history.history['val_classification_output_loss'],
                   label='Валидационные', linewidth=2)
    axes[0, 1].set_title('Потери классификации', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Эпоха')
    axes[0, 1].set_ylabel('Потери')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # График 3: Потери автокодировщика
    axes[1, 0].plot(history.history['decoder_output_loss'],
                   label='Обучающие', linewidth=2)
    axes[1, 0].plot(history.history['val_decoder_output_loss'],
                   label='Валидационные', linewidth=2)
    axes[1, 0].set_title('Потери автокодировщика (MSE)', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Эпоха')
    axes[1, 0].set_ylabel('MSE')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # График 4: Общая потеря
    axes[1, 1].plot(history.history['loss'],
                   label='Общие обучающие', linewidth=2)
    axes[1, 1].plot(history.history['val_loss'],
                   label='Общие валидационные', linewidth=2)
    axes[1, 1].set_title('Общие потери', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Эпоха')
    axes[1, 1].set_ylabel('Потери')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.suptitle('РЕЗУЛЬТАТЫ ОБУЧЕНИЯ', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

plot_training_results(history)

In [ ]:
# 10. ВИЗУАЛИЗАЦИЯ РАБОТЫ АВТОКОДИРОВЩИКА
print("\n" + "="*60)
print("ВИЗУАЛИЗАЦИЯ РАБОТЫ АВТОКОДИРОВЩИКА")
print("="*60)

# Получаем один батч изображений
sample_batch = next(train_generator)
sample_images = sample_batch[0][:6]

# Получаем выход декодера
decoded_images = model.predict(sample_images)[0]  # Первый выход - декодер

# Визуализация
plt.figure(figsize=(12, 8))
for i in range(6):
    # Оригинальное изображение
    ax = plt.subplot(3, 6, i + 1)
    plt.imshow(sample_images[i])
    plt.title("Оригинал" if i == 0 else "")
    plt.axis('off')

    # Восстановленное изображение
    ax = plt.subplot(3, 6, i + 7)
    plt.imshow(decoded_images[i])
    plt.title("Восстановлено" if i == 0 else "")
    plt.axis('off')

    # Разница
    ax = plt.subplot(3, 6, i + 13)
    difference = np.abs(sample_images[i] - decoded_images[i])
    plt.imshow(difference)
    plt.title("Разница" if i == 0 else "")
    plt.axis('off')

plt.suptitle('Работа автокодировщика: оригинал vs восстановление',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 11. ВИЗУАЛИЗАЦИЯ ПРЕДСКАЗАНИЙ
print("\n" + "="*60)
print("ВИЗУАЛИЗАЦИЯ ПРЕДСКАЗАНИЙ НА ТЕСТОВЫХ ДАННЫХ")
print("="*60)

# Получаем батч тестовых данных
test_images, test_labels = next(test_generator)

# Делаем предсказания
predictions = classification_model.predict(test_images[:12])

# Визуализируем результаты
class_names = ["Auto Rickshaws", "Bikes", "Cars", "Motorcycles",
               "Planes", "Ships", "Trains"]

plt.figure(figsize=(15, 10))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(test_images[i])

    true_class = np.argmax(test_labels[i])
    pred_class = np.argmax(predictions[i])
    confidence = np.max(predictions[i])

    true_name = class_names[true_class]
    pred_name = class_names[pred_class]

    color = 'green' if true_class == pred_class else 'red'

    plt.title(f"True: {true_name[:10]}\nPred: {pred_name[:10]}\nConf: {confidence:.2f}",
              color=color, fontsize=9)
    plt.axis('off')

plt.suptitle('Примеры предсказаний на тестовых данных', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 12. АНАЛИЗ РЕЗУЛЬТАТОВ
print("\n" + "="*60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ ОБУЧЕНИЯ")
print("="*60)

# Получаем лучшие результаты
best_val_acc = max(history.history['val_classification_output_accuracy'])
best_val_loss = min(history.history['val_classification_output_loss'])
final_train_acc = history.history['classification_output_accuracy'][-1]
final_val_acc = history.history['val_classification_output_accuracy'][-1]

print(f"Лучшая валидационная точность: {best_val_acc:.4f}")
print(f"Лучшие валидационные потери: {best_val_loss:.4f}")
print(f"Финальная обучающая точность: {final_train_acc:.4f}")
print(f"Финальная валидационная точность: {final_val_acc:.4f}")
print(f"Финальная тестовая точность: {test_results[1]:.4f}")

# Анализ переобучения
overfitting_gap = final_train_acc - final_val_acc
print(f"\nРазница между train и val точностью: {overfitting_gap:.4f}")

if overfitting_gap > 0.15:
    print("СТАТУС: Сильное переобучение!")
elif overfitting_gap > 0.05:
    print("СТАТУС: Умеренное переобучение")
else:
    print("СТАТУС: Хорошая обобщающая способность")

# Анализ эффективности автокодировщика
final_autoencoder_loss = history.history['decoder_output_loss'][-1]
final_autoencoder_val_loss = history.history['val_decoder_output_loss'][-1]
print(f"\nПотери автокодировщика (train): {final_autoencoder_loss:.4f}")
print(f"Потери автокодировщика (val): {final_autoencoder_val_loss:.4f}")